In [2]:
import pandas as pd
import plotly
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pickle

from utils.datautils import Readdataset, calculate_dataset_metrics, Splitview
from utils.train_utils import Evaluate_model

In [12]:
model_path = '../Tree_Models/BeetleFly_tree.pkl'
model = pickle.load(open(model_path, 'rb'))

In [13]:
model

{0: <utils.train_utils.Node at 0x12d66f100>,
 1: <utils.train_utils.Node at 0x12d66efd0>,
 2: <utils.train_utils.Node at 0x12d66ed70>,
 3: <utils.train_utils.Node at 0x12d66eb10>,
 4: <utils.train_utils.Node at 0x12d66ec40>}

In [16]:
model[2].bestmodel

TL_NN1()

In [21]:
def print_tree_structure(tree):
    """
    Print the structure of the NSTSC tree and details of each node.
    
    Parameters:
    -----------
    tree : dict
        Dictionary representing the NSTSC tree model
    """
    import torch
    from Models_node import TL_NN1, TL_NN2, TL_NN3, TL_NN4
    
    def get_node_type(model):
        """Identify the type of neural network model"""
        if isinstance(model, TL_NN1):
            return "TL_NN1 (Conjunction/AND)"
        elif isinstance(model, TL_NN2):
            return "TL_NN2 (Disjunction/OR)"
        elif isinstance(model, TL_NN3):
            return "TL_NN3 (Always/Globally)"
        elif isinstance(model, TL_NN4):
            return "TL_NN4 (Eventually/Finally)"
        else:
            return "Unknown Model"
    
    def print_node_info(node_id, node, prefix="", is_last=True):
        # Print node information
        connector = "└── " if is_last else "├── "
        print(f"{prefix}{connector}Node {node_id}")
        
        next_prefix = prefix + ("    " if is_last else "│   ")
        
        # Print predicted class if available
        if hasattr(node, 'predcls'):
            print(f"{next_prefix}Predicted Class: {node.predcls}")
        
        # Print best model class if available
        if hasattr(node, 'bstmdlclass'):
            print(f"{next_prefix}Best Model Class: {node.bstmdlclass}")
            
        # Print best model info if available
        if hasattr(node, 'bestmodel'):
            model_type = get_node_type(node.bestmodel)
            print(f"{next_prefix}Best Model: {model_type}")
            
        # Print Gini index if available
        if hasattr(node, 'ginis'):
            print(f"{next_prefix}Gini Index: {node.ginis:.6f}")
            
        # Print children recursively
        has_left = hasattr(node, 'leftchd')
        has_right = hasattr(node, 'rightchd')
        
        if has_left:
            print_node_info(node.leftchd, tree[node.leftchd], next_prefix, not has_right)
        
        if has_right:
            print_node_info(node.rightchd, tree[node.rightchd], next_prefix, True)
    
    # Start printing from root (node 0)
    print("NSTSC Tree Structure:")
    print("====================\n")
    print_node_info(0, tree[0])
    
    # Print summary of the tree
    print("\nTree Summary:")
    print(f"Total Nodes: {len(tree)}")
    
    # Count leaf nodes
    leaf_nodes = sum(1 for node_id, node in tree.items() 
                   if not hasattr(node, 'leftchd') and not hasattr(node, 'rightchd'))
    print(f"Leaf Nodes: {leaf_nodes}")
    print(f"Internal Nodes: {len(tree) - leaf_nodes}")

In [22]:
# Print tree structure for the BeetleFly model
print_tree_structure(model)

NSTSC Tree Structure:

└── Node 0
    Predicted Class: 1
    Best Model Class: 2
    Best Model: TL_NN1 (Conjunction/AND)
    Gini Index: 0.090909
    ├── Node 1
    │   Predicted Class: 2
    │   Gini Index: 0.000000
    └── Node 2
        Predicted Class: 1
        Best Model Class: 2
        Best Model: TL_NN1 (Conjunction/AND)
        Gini Index: 0.000000
        ├── Node 3
        │   Predicted Class: 2
        │   Gini Index: 0.000000
        └── Node 4
            Predicted Class: 1
            Gini Index: 0.000000

Tree Summary:
Total Nodes: 5
Leaf Nodes: 3
Internal Nodes: 2


In [19]:
# Function to load and analyze other models
def load_and_analyze_model(model_name):
    import pickle
    import os
    
    model_path = f'../Tree_Models/{model_name}_tree.pkl'
    if os.path.exists(model_path):
        model = pickle.load(open(model_path, 'rb'))
        print(f"\n\nAnalyzing model for dataset: {model_name}")
        print("=" * (len(model_name) + 24))
        print_tree_structure(model)
        return model
    else:
        print(f"Model file {model_path} not found.")
        return None

# Try loading and analyzing other models - uncomment to use
# other_models = ['Coffee', 'Meat']
# for model_name in other_models:
#     load_and_analyze_model(model_name)

In [20]:
load_and_analyze_model('Coffee')



Analyzing model for dataset: Coffee
NSTSC Tree Structure:

└── Node 0
    Predicted Class: 0
    Best Model Class: 0
    Best Model: TL_NN1 (Conjunction/AND)
    Gini Index: 0.126050
    ├── Node 1
    │   Predicted Class: 0
    │   Gini Index: 0.000000
    └── Node 2
        Predicted Class: 1
        Gini Index: 0.000000

Tree Summary:
Total Nodes: 3
Leaf Nodes: 2
Internal Nodes: 1


{0: <utils.train_utils.Node at 0x12d66d810>,
 1: <utils.train_utils.Node at 0x12d66e3f0>,
 2: <utils.train_utils.Node at 0x12d66e520>}

In [23]:
# Inspect Node 2 properties
print("Node 2 Properties:")
for attr in dir(model[2]):
    if not attr.startswith('__') and not callable(getattr(model[2], attr)):
        try:
            value = getattr(model[2], attr)
            print(f"{attr}: {value}")
        except:
            print(f"{attr}: [Unable to display]")

print("\nChecking stoptrain flag:")
print(f"Node 2 stoptrain: {getattr(model[2], 'stoptrain', 'Not set')}")


Node 2 Properties:
Testidx: [ 0  1  2  5  7  8 10 14 15 18 19]
Xpreds: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Xpredsupto: [0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0.]
bstmdlclass: 2
childtype: rightchild
falseidx: [ 0  1  3  5  8  9 11 12 14 18]
falseidxt: [ 0  1  2  5  7  8 10 15 18 19]
ginis: 0.0
ginist: 0.16528925619834722
idx: 2
leftchd: 3
predcls: 1
prntnb: 0
rightchd: 4
stoptrain: False
testaccuupto: 0.0
testfalseidx: [ 0  1  2  3  4  5  6  8  9 10]
testidx: [ 0  1  2  5  7  8 10 14 15 18 19]
testtrueidx: [7]
trainidx: [ 0  1  2  3  5  8  9 11 12 14 18]
trueidx: [2]
trueidxt: [14]
ycount: [ 0. 10.  1.]
ycountt: [ 0. 10.  1.]

Checking stoptrain flag:
Node 2 stoptrain: False


In [24]:
# Let's add a modified version of the Updateleftchd and Updaterigtchd functions 
# to understand why Node 2 still split despite having ginis=0

def examine_split_decision(node_id):
    node = model[node_id]
    print(f"Examining Node {node_id}:")
    print(f"  ginis: {getattr(node, 'ginis', 'Not set')}")
    print(f"  ginist: {getattr(node, 'ginist', 'Not set')}")
    print(f"  stoptrain: {getattr(node, 'stoptrain', 'Not set')}")
    
    # In train_utils.py, the condition for setting stoptrain=True is:
    # if ylginit == 0 or ylgini == 0:
    #     Nodes[maxnum].stoptrain = True
    
    print("\nAccording to the code logic:")
    if hasattr(node, 'ginis') and hasattr(node, 'ginist'):
        should_stop = (node.ginis == 0 or node.ginist == 0)
        print(f"  Should stop training based on gini values: {should_stop}")
        print(f"  Actual stoptrain value: {getattr(node, 'stoptrain', 'Not set')}")
        if should_stop != getattr(node, 'stoptrain', False):
            print("  ⚠️ INCONSISTENCY: stoptrain flag doesn't match expected value based on Gini indices")
    
    # Check if the node was split despite stoptrain=True
    has_children = hasattr(node, 'leftchd') or hasattr(node, 'rightchd')
    if getattr(node, 'stoptrain', False) and has_children:
        print("  ⚠️ INCONSISTENCY: Node was split despite stoptrain=True")
    elif not getattr(node, 'stoptrain', True) and not has_children:
        print("  ℹ️ Node was not split despite stoptrain=False")

# Examine Node 2
examine_split_decision(2)

# Also examine nodes 3 and 4 to see their properties
print("\n" + "-"*50 + "\n")
examine_split_decision(3)
print("\n" + "-"*50 + "\n")
examine_split_decision(4)

Examining Node 2:
  ginis: 0.0
  ginist: 0.16528925619834722
  stoptrain: False

According to the code logic:
  Should stop training based on gini values: True
  Actual stoptrain value: False
  ⚠️ INCONSISTENCY: stoptrain flag doesn't match expected value based on Gini indices

--------------------------------------------------

Examining Node 3:
  ginis: 0.0
  ginist: 0.0
  stoptrain: True

According to the code logic:
  Should stop training based on gini values: True
  Actual stoptrain value: True

--------------------------------------------------

Examining Node 4:
  ginis: 0.0
  ginist: 0.0
  stoptrain: True

According to the code logic:
  Should stop training based on gini values: True
  Actual stoptrain value: True


In [25]:
# Explanation of why Node 2 was split despite ginis = 0

print("Explanation of the Node 2 inconsistency:")
print("\nIn the NSTSC algorithm, the stoptrain flag is set during node creation in Updateleftchd and Updaterigtchd.")
print("However, Node 2 was created before its Gini index was calculated and set to 0.")
print("\nSequence of events:")
print("  1. Node 0 is processed and split into Node 1 and Node 2")
print("  2. Node 1 is processed with no children (stoptrain=True)")
print("  3. Node 2 is processed and trained with bestmodel")
print("  4. After training, Node 2's ginis is set to 0")
print("  5. However, the node was already created with stoptrain=False")
print("  6. Build_tree continues to process Node 2, creating Nodes 3 and 4")
print("\nKey insight: The stoptrain flag was set BEFORE training the node,")
print("but the ginis value is set AFTER training. There's no code that updates")
print("stoptrain after the Gini index is determined from model training.")

print("\nLet's review the code flow:")
print("  1. Train model on Node 2 → Sets ginis to 0")
print("  2. No code updates stoptrain based on the trained model's Gini index")
print("  3. Tree building continues processing Node 2 since stoptrain is still False")

Explanation of the Node 2 inconsistency:

In the NSTSC algorithm, the stoptrain flag is set during node creation in Updateleftchd and Updaterigtchd.
However, Node 2 was created before its Gini index was calculated and set to 0.

Sequence of events:
  1. Node 0 is processed and split into Node 1 and Node 2
  2. Node 1 is processed with no children (stoptrain=True)
  3. Node 2 is processed and trained with bestmodel
  4. After training, Node 2's ginis is set to 0
  5. However, the node was already created with stoptrain=False
  6. Build_tree continues to process Node 2, creating Nodes 3 and 4

Key insight: The stoptrain flag was set BEFORE training the node,
but the ginis value is set AFTER training. There's no code that updates
stoptrain after the Gini index is determined from model training.

Let's review the code flow:
  1. Train model on Node 2 → Sets ginis to 0
  2. No code updates stoptrain based on the trained model's Gini index
  3. Tree building continues processing Node 2 since

In [27]:
# Simple tree visualization with detailed node information
def simple_tree_viz(tree):
    print("NSTSC Tree Structure:")
    print("===================")
    
    def print_node(node_id, indent=""):
        node = tree[node_id]
        print(f"{indent}Node {node_id}:")
        indent2 = indent + "  "
        
        # Print node attributes
        attrs = ["predcls", "bstmdlclass", "ginis", "ginist", "stoptrain"]
        for attr in attrs:
            if hasattr(node, attr):
                value = getattr(node, attr)
                if attr in ["ginis", "ginist"] and isinstance(value, (int, float)):
                    print(f"{indent2}{attr}: {value:.6f}")
                else:
                    print(f"{indent2}{attr}: {value}")
        
        # Print model info
        if hasattr(node, 'bestmodel'):
            from Models_node import TL_NN1, TL_NN2, TL_NN3, TL_NN4
            if isinstance(node.bestmodel, TL_NN1):
                model_name = "TL_NN1 (Conjunction/AND)"
            elif isinstance(node.bestmodel, TL_NN2):
                model_name = "TL_NN2 (Disjunction/OR)"
            elif isinstance(node.bestmodel, TL_NN3):
                model_name = "TL_NN3 (Always/Globally)"
            elif isinstance(node.bestmodel, TL_NN4):
                model_name = "TL_NN4 (Eventually/Finally)"
            else:
                model_name = "Unknown Model"
            print(f"{indent2}bestmodel: {model_name}")
    
    # Print all nodes with their connections
    for node_id in sorted(tree.keys()):
        print_node(node_id)
        print()
    
    # Print tree structure
    print("Tree Structure:")
    print("-" * 15)
    
    def print_children(node_id, indent=""):
        node = tree[node_id]
        print(f"{indent}Node {node_id}")
        next_indent = indent + "│   "
        last_indent = indent + "    "
        
        if hasattr(node, 'leftchd') and hasattr(node, 'rightchd'):
            print(f"{indent}├── True → Node {node.leftchd}")
            print_children(node.leftchd, next_indent)
            print(f"{indent}└── False → Node {node.rightchd}")
            print_children(node.rightchd, last_indent)
        elif hasattr(node, 'leftchd'):
            print(f"{indent}└── True → Node {node.leftchd}")
            print_children(node.leftchd, last_indent)
        elif hasattr(node, 'rightchd'):
            print(f"{indent}└── False → Node {node.rightchd}")
            print_children(node.rightchd, last_indent)
        else:
            # Leaf node
            print(f"{indent}└── Leaf (predcls: {node.predcls})")
    
    print_children(0)

# Run the simple tree visualization
simple_tree_viz(model)

NSTSC Tree Structure:
Node 0:
  predcls: 1
  bstmdlclass: 2
  ginis: 0.09090904146432877
  stoptrain: False
  bestmodel: TL_NN1 (Conjunction/AND)

Node 1:
  predcls: 2
  ginis: 0.000000
  ginist: 0.000000
  stoptrain: True

Node 2:
  predcls: 1
  bstmdlclass: 2
  ginis: 0.0
  ginist: 0.165289
  stoptrain: False
  bestmodel: TL_NN1 (Conjunction/AND)

Node 3:
  predcls: 2
  ginis: 0.000000
  ginist: 0.000000
  stoptrain: True

Node 4:
  predcls: 1
  ginis: 0.000000
  ginist: 0.000000
  stoptrain: True

Tree Structure:
---------------
Node 0
├── True → Node 1
│   Node 1
│   └── Leaf (predcls: 2)
└── False → Node 2
    Node 2
    ├── True → Node 3
    │   Node 3
    │   └── Leaf (predcls: 2)
    └── False → Node 4
        Node 4
        └── Leaf (predcls: 1)


In [28]:
# Examine the data distribution in nodes 2, 3, and 4

def analyze_node_data_distribution(node_id):
    node = model[node_id]
    print(f"Node {node_id} Analysis:")
    print(f"  Predicted Class (predcls): {getattr(node, 'predcls', 'N/A')}")
    print(f"  Best Model Class (bstmdlclass): {getattr(node, 'bstmdlclass', 'N/A')}")
    print(f"  Gini Index (ginis): {getattr(node, 'ginis', 'N/A')}")
    print(f"  Validation Gini Index (ginist): {getattr(node, 'ginist', 'N/A')}")
    
    # Check class distribution in the node
    if hasattr(node, 'ycount'):
        print(f"  Training Data Class Distribution (ycount): {node.ycount}")
    if hasattr(node, 'ycountt'):
        print(f"  Validation Data Class Distribution (ycountt): {node.ycountt}")
    
    # Check indices of data points in this node
    print(f"  Training Indices: {len(getattr(node, 'trainidx', []))} points")
    print(f"  Test/Validation Indices: {len(getattr(node, 'testidx', []))} points")
    
    # For nodes with children, check split points
    if hasattr(node, 'trueidx') and hasattr(node, 'falseidx'):
        print(f"  True branch (trueidx): {len(node.trueidx)} training points")
        print(f"  False branch (falseidx): {len(node.falseidx)} training points")
    if hasattr(node, 'trueidxt') and hasattr(node, 'falseidxt'):
        print(f"  True branch validation (trueidxt): {len(node.trueidxt)} validation points")
        print(f"  False branch validation (falseidxt): {len(node.falseidxt)} validation points")
    
    print("\n")

print("Analyzing Node Data Distributions:\n")
analyze_node_data_distribution(2)
analyze_node_data_distribution(3)
analyze_node_data_distribution(4)

print("==== Detailed Class Distributions ====\n")

# Check if the sum of child node class distributions matches parent node
print("Checking consistency between parent and child nodes:")
if hasattr(model[3], 'ycount') and hasattr(model[4], 'ycount') and hasattr(model[2], 'ycount'):
    total_children = model[3].ycount + model[4].ycount
    print(f"  Node 2 class distribution: {model[2].ycount}")
    print(f"  Sum of Nodes 3&4 distributions: {total_children}")
    print(f"  Match: {np.array_equal(model[2].ycount, total_children)}")

# Get predcls logic from train_utils.py
print("\nPredcls Assignment Logic:")
print("  Node predcls is determined from yoricountt.argmax() - the most common class in validation data")
print("  This is assigned before the Gini index is calculated or any split occurs")
print("  In other words, the predcls is assigned based on the node's validation data distribution")

# Check True/False split data details
print("\nSplit Details for Node 2:")
if hasattr(model[2], 'trueidx') and hasattr(model[2], 'falseidx'):
    print(f"  Training data in true branch (Node 3): {len(model[2].trueidx)} points")
    print(f"  Training data in false branch (Node 4): {len(model[2].falseidx)} points")
    print(f"  Total training data in Node 2: {len(model[2].trainidx)} points")
    if len(model[2].trueidx) + len(model[2].falseidx) != len(model[2].trainidx):
        print("  ⚠️ INCONSISTENCY: Sum of branch data does not match total data")
    
    # Check if all points go to one branch (which would contradict the zero Gini index)
    if len(model[2].trueidx) == 0 or len(model[2].falseidx) == 0:
        print("  ⚠️ One branch has no data points, which is consistent with Gini=0")
    else:
        print("  ⚠️ Both branches have data points, which is INCONSISTENT with Gini=0")


Analyzing Node Data Distributions:

Node 2 Analysis:
  Predicted Class (predcls): 1
  Best Model Class (bstmdlclass): 2
  Gini Index (ginis): 0.0
  Validation Gini Index (ginist): 0.16528925619834722
  Training Data Class Distribution (ycount): [ 0. 10.  1.]
  Validation Data Class Distribution (ycountt): [ 0. 10.  1.]
  Training Indices: 11 points
  Test/Validation Indices: 11 points
  True branch (trueidx): 1 training points
  False branch (falseidx): 10 training points
  True branch validation (trueidxt): 1 validation points
  False branch validation (falseidxt): 10 validation points


Node 3 Analysis:
  Predicted Class (predcls): 2
  Best Model Class (bstmdlclass): N/A
  Gini Index (ginis): 0.0
  Validation Gini Index (ginist): 0.0
  Training Data Class Distribution (ycount): [0. 0. 1.]
  Validation Data Class Distribution (ycountt): [0. 0. 1.]
  Training Indices: 1 points
  Test/Validation Indices: 1 points


Node 4 Analysis:
  Predicted Class (predcls): 1
  Best Model Class (bstm

In [29]:
# Inspect the actual data points that go into true and false branches
print("Class Distribution Analysis for Node 2 Split:")

# Get the actual classes of points in true and false branches
def check_class_distribution_in_branches(node_id):
    node = model[node_id]
    
    # Check training data
    if hasattr(node, 'trainidx') and hasattr(node, 'trueidx') and hasattr(node, 'falseidx'):
        print(f"\nNode {node_id} Training Data:")
        train_idx = node.trainidx
        true_idx = node.trueidx
        false_idx = node.falseidx
        
        print(f"  All points indices: {train_idx}")
        print(f"  True branch indices: {true_idx}")
        print(f"  False branch indices: {false_idx}")
        
        if hasattr(node, 'bstmdlclass'):
            print(f"\n  Best model class (bstmdlclass): {node.bstmdlclass}")
            
            # This checks the explanation for binary encoding in Ecdlabel function
            print(f"  Note: bstmdlclass = {node.bstmdlclass} means this model is trained to separate")
            print(f"        class {node.bstmdlclass} (value 1) from all other classes (value 0)")
        
    # Check validation data
    if hasattr(node, 'testidx') and hasattr(node, 'trueidxt') and hasattr(node, 'falseidxt'):
        print(f"\nNode {node_id} Validation Data:")
        test_idx = node.testidx
        true_idx = node.trueidxt
        false_idx = node.falseidxt
        
        print(f"  All points indices: {test_idx}")
        print(f"  True branch indices: {true_idx}")
        print(f"  False branch indices: {false_idx}")

check_class_distribution_in_branches(2)

# Let's investigate further based on what we've learned
print("\nExplanation for Different Predicted Classes:")
print("1. In Node 2, we have 11 data points: 10 of class 1 and 1 of class 2")
print("2. Node 2's best model (TL_NN1) is trained to separate class 2 from others")
print("3. The model successfully separates the single class 2 point (into Node 3) from all the class 1 points (into Node 4)")
print("4. This perfect separation gives a training Gini index of 0 in Node 2")
print("5. Node 3 contains only class 2 data, so its predcls=2")
print("6. Node 4 contains only class 1 data, so its predcls=1")
print("\nLooking at the code in Trainnode() function in train_utils.py:")
print("predcls is assigned BEFORE model training by: Nodes[pronum].predcls = yoricountt.argmax()")
print("bstmdlclass is assigned DURING model training when finding best model")
print("\nConclusion:")
print("The different predcls values in Nodes 3 and 4 reflect the perfect class separation achieved by Node 2's model.")
print("Node 2's Gini=0 means that its best model (separating class 2 from others) perfectly")
print("split the data points into pure class distributions in each child node.")


Class Distribution Analysis for Node 2 Split:

Node 2 Training Data:
  All points indices: [ 0  1  2  3  5  8  9 11 12 14 18]
  True branch indices: [2]
  False branch indices: [ 0  1  3  5  8  9 11 12 14 18]

  Best model class (bstmdlclass): 2
  Note: bstmdlclass = 2 means this model is trained to separate
        class 2 (value 1) from all other classes (value 0)

Node 2 Validation Data:
  All points indices: [ 0  1  2  5  7  8 10 14 15 18 19]
  True branch indices: [14]
  False branch indices: [ 0  1  2  5  7  8 10 15 18 19]

Explanation for Different Predicted Classes:
1. In Node 2, we have 11 data points: 10 of class 1 and 1 of class 2
2. Node 2's best model (TL_NN1) is trained to separate class 2 from others
3. The model successfully separates the single class 2 point (into Node 3) from all the class 1 points (into Node 4)
4. This perfect separation gives a training Gini index of 0 in Node 2
5. Node 3 contains only class 2 data, so its predcls=2
6. Node 4 contains only class 1 d

## Why Node 2 with Gini=0 has Children with Different Predicted Classes

We've uncovered the explanation for this apparent contradiction. The Gini index of 0 at Node 2 and the different predicted classes at Nodes 3 and 4 are actually consistent with how the NSTSC algorithm works:

### Key Insights:

1. **Node 2 Data Composition**: Node 2 contains 11 data points: 10 of class 1 and 1 of class 2

2. **Binary Classification at Each Node**: The NSTSC model trains a binary classifier at each node. In Node 2, the best model is a TL_NN1 (Conjunction/AND) model trained specifically to separate class 2 from all others.

3. **Perfect Separation Achieved**: This model perfectly separates the single class 2 instance (sending it to Node 3) from all the class 1 instances (sending them to Node 4). This is why the Gini index is 0 - the model has achieved a perfect separation.

4. **Different Predicted Classes**: 
   - Node 3 contains only class 2 data, so `predcls=2`
   - Node 4 contains only class 1 data, so `predcls=1`

### How the Gini Index Works in NSTSC:

The Gini index of 0 in Node 2 doesn't mean that all data points belong to the same class. Instead, it means:

- The **splitting rule** (the neural network model) perfectly separates the classes
- After the split, each child node contains data from only one class
- The weighted average impurity of the child nodes is 0

The `bstmdlclass` value of 2 for Node 2 tells us which class the model is trying to separate from the rest. It means "this model separates class 2 (positive) from all other classes (negative)." When this separation is perfect (as in this case), the Gini index becomes 0.

### Conclusion:

The different `predcls` values in Nodes 3 and 4 actually confirm that Node 2's split was perfect, which is consistent with its Gini index of 0. Rather than being a contradiction, this is exactly what we would expect from a perfect binary classifier that has successfully separated the classes.

## Understanding NSTSC Node Attributes

### Class Distribution Attributes

**`ycount` and `ycountt`:**
- **`ycount`**: Count of training data samples in each class for the current node
- **`ycountt`**: Count of validation/test data samples in each class for the current node

These arrays show how many samples of each class are present in the node. For example, if we have 3 classes (0, 1, 2), then `ycount = [5, 10, 2]` means there are 5 samples of class 0, 10 samples of class 1, and 2 samples of class 2 in this node's training data.

**`yoricount` and `yoricountt`:**
- **`yoricount`**: Original count of training samples per class before processing in the current node
- **`yoricountt`**: Original count of validation/test samples per class before processing in the current node

These are temporary variables used during node training in the `Trainnode()` function and are not stored as node attributes. They represent the class distribution before any splitting occurs.

### Classification Attributes

**`predcls` vs `bstmdlclass`:**

**`predcls` (Predicted Class):**
- The majority class in the node's validation data
- Determined by: `node.predcls = yoricountt.argmax()`
- Used as the final prediction if this becomes a leaf node
- Assigned BEFORE any model training occurs
- Based purely on class frequency, not model performance

**`bstmdlclass` (Best Model Class):**
- The class that the node's neural network model is trained to separate from others
- Only exists for internal nodes with a trained model
- Determined during model training when finding the split with minimum Gini index
- Represents which class the binary classifier is focused on (the "positive" class)
- When `bstmdlclass = 2`, the model separates class 2 (positive) from all others (negative)

### Important Distinction:

1. **`predcls`** is what the node would predict if it became a leaf node (based on majority class)
2. **`bstmdlclass`** is what the node's splitting model is trained to identify (based on best Gini improvement)

In a perfect split, the children nodes will have different `predcls` values that reflect the classes being separated by the parent node's model (which uses `bstmdlclass` to determine the split).

In [30]:
# Demonstrating ycount, ycountt, predcls, and bstmdlclass with examples from our tree

# Print header and formatting helper
def format_node_info(node_id, show_detail=False):
    node = model[node_id]
    has_children = hasattr(node, 'leftchd') or hasattr(node, 'rightchd')
    node_type = "Internal" if has_children else "Leaf"
    
    print(f"\n{'=' * 50}")
    print(f"Node {node_id} ({node_type} Node) Details:")
    print(f"{'-' * 30}")
    
    # Show class distribution
    if hasattr(node, 'ycount'):
        print(f"ycount (training class distribution): {node.ycount}")
    if hasattr(node, 'ycountt'):
        print(f"ycountt (validation class distribution): {node.ycountt}")
        
    # Show predicted class and best model class
    if hasattr(node, 'predcls'):
        print(f"\npredcls (majority class for prediction): {node.predcls}")
        # Verify that predcls is the argmax of ycountt
        if hasattr(node, 'ycountt'):
            argmax_class = np.argmax(node.ycountt)
            print(f"Verification: argmax of ycountt = {argmax_class} {'✓' if argmax_class == node.predcls else '✗'}")
    
    if hasattr(node, 'bstmdlclass'):
        print(f"\nbstmdlclass (class to separate): {node.bstmdlclass}")
        print(f"This model separates class {node.bstmdlclass} from all others")
    
    # Show model type if available
    if hasattr(node, 'bestmodel'):
        from Models_node import TL_NN1, TL_NN2, TL_NN3, TL_NN4
        if isinstance(node.bestmodel, TL_NN1):
            model_type = "TL_NN1 (Conjunction/AND)"
        elif isinstance(node.bestmodel, TL_NN2):
            model_type = "TL_NN2 (Disjunction/OR)"
        elif isinstance(node.bestmodel, TL_NN3):
            model_type = "TL_NN3 (Always/Globally)"
        elif isinstance(node.bestmodel, TL_NN4):
            model_type = "TL_NN4 (Eventually/Finally)"
        else:
            model_type = "Unknown Model"
        print(f"Best Model: {model_type}")
        
    # Show children if available
    if hasattr(node, 'leftchd'):
        print(f"\nLeft Child: Node {node.leftchd} (True branch)")
        if hasattr(model[node.leftchd], 'predcls'):
            print(f"  - predcls: {model[node.leftchd].predcls}")
    if hasattr(node, 'rightchd'):
        print(f"Right Child: Node {node.rightchd} (False branch)")
        if hasattr(model[node.rightchd], 'predcls'):
            print(f"  - predcls: {model[node.rightchd].predcls}")
    
    # Show more detailed data if requested
    if show_detail:
        print("\nDetailed Data:")
        if hasattr(node, 'trainidx'):
            print(f"Training indices: {node.trainidx[:5]}{'...' if len(node.trainidx) > 5 else ''}")
        if hasattr(node, 'trueidx') and hasattr(node, 'falseidx'):
            print(f"True branch indices: {node.trueidx[:5]}{'...' if len(node.trueidx) > 5 else ''}")
            print(f"False branch indices: {node.falseidx[:5]}{'...' if len(node.falseidx) > 5 else ''}")

# Process all nodes in our model
for node_id in sorted(model.keys()):
    format_node_info(node_id)

# Show a diagram explaining predcls vs bstmdlclass
print("\n\n" + "=" * 80)
print("Conceptual Difference between predcls and bstmdlclass:")
print("-" * 80)
print("predcls: WHAT the node predicts if it's a leaf (majority class in validation data)")
print("bstmdlclass: HOW the node splits data (which class it separates from others)")
print("=" * 80)


Node 0 (Internal Node) Details:
------------------------------
ycount (training class distribution): [ 0. 10. 10.]

predcls (majority class for prediction): 1

bstmdlclass (class to separate): 2
This model separates class 2 from all others
Best Model: TL_NN1 (Conjunction/AND)

Left Child: Node 1 (True branch)
  - predcls: 2
Right Child: Node 2 (False branch)
  - predcls: 1

Node 1 (Leaf Node) Details:
------------------------------
ycount (training class distribution): [0. 0. 9.]
ycountt (validation class distribution): [0. 0. 9.]

predcls (majority class for prediction): 2
Verification: argmax of ycountt = 2 ✓

Node 2 (Internal Node) Details:
------------------------------
ycount (training class distribution): [ 0. 10.  1.]
ycountt (validation class distribution): [ 0. 10.  1.]

predcls (majority class for prediction): 1
Verification: argmax of ycountt = 1 ✓

bstmdlclass (class to separate): 2
This model separates class 2 from all others
Best Model: TL_NN1 (Conjunction/AND)

Left Chi